In [1]:
from datetime import datetime

import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import metpy.constants as mpconst
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pint
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.ndimage import gaussian_filter
import xarray as xr

In [2]:
from datetime import datetime

import metpy.calc as mpcalc
import metpy.constants as mpconst
from metpy.units import units
import numpy as np
import pint
import xarray as xr

In [3]:
def read_gdlist(name):
    file = open(name, 'rb')

    for _ in range(6):
        file.readline()

    grid_row = file.readline()
    nlon = int(grid_row[-11:-5].strip())
    nlat = int(grid_row[-4:].strip())
    #print(nlon,nlat)

    gempak_array = np.empty((nlat,nlon))

    for _ in range(2):
        file.readline()

    scale_row = file.readline()
    scale_factor = int(scale_row[19:].strip())
    #print(scale_factor)

    for _ in range(92):
        file.readline()

    col = 0
    row = 0
    for line in file.readlines():
        if (line[:13].strip() != b'') & (col >= nlon):
            col = 0
            row+=1
        for count in [0,1,2,3,4,5,6,7]:
            gempak_array[row,col] = line[8+count*9:17+count*9]
            col+=1

    file.close()
    
    return (gempak_array, -scale_factor)

In [66]:
def error_stats(metpy_data, gempak_data):
    ignore = (gempak_data != -9999.0) * ~np.isnan(metpy_data)
    
    if isinstance(metpy_data.data, pint.Quantity):
        print(f'Is Xarray with Quantity? True')
        mdata = metpy_data.values[ignore]
    elif isinstance(metpy_data, pint.Quantity):
        print(f'Is Xarray with Quantity? False')
        print(f'Is Metpy Unit Array? True')
        mdata = metpy_data.m[ignore]
    else:
        print(f'Is Xarray with Quantity? False')
        print(f'Is Metpy Unit Array? False')
        print(f'I got back just a numpy array')
        try:
            mdata = metpy_data.values[ignore]
        except:
            mdata = metpy_data[ignore]
    gempak_data = gempak_data[ignore]
    
    print()
    print('Mean Comparison')
    print(f'  Mean Values (MetPy): {mdata.mean()}')
    print(f'  Mean Values (GEMPAK): {gempak_data.mean()}')
    print()
    print('Max Comparison')
    print(f'  Max Values (MetPy): {mdata.max()}')
    print(f'  Max Values (GEMPAK): {gempak_data.max()}')
    print()
    print('Min Comparison')
    print(f'  Min Values (MetPy): {mdata.min()}')
    print(f'  Min Values (GEMPAK): {gempak_data.min()}')

    print()
    print('Difference Array')
    diff = mdata - gempak_data
    print(diff)
    print()
    print('Various Statistical Analyses')
    print(f'  Average Absolute Difference: {np.nanmean(np.abs(diff))}')
    print(f'  RMS Error: {np.sqrt(np.nansum(diff**2))/len(diff.ravel())}')
    print(f'  Standard Deviation of Difference: {np.nanstd(diff)}')
    print(f'  Max Diff: {np.max(diff)}')
    print(f'  Min Diff: {np.min(diff)}')
    print(f'  Correlation: {np.corrcoef(mdata.ravel(), gempak_data.ravel())[0][1]}')
    print(f'  Relative Magnitude Difference: {np.nanmean(np.abs(diff))/np.nanmax(mdata)}')
    print()

In [5]:
#gfs_data = xr.open_dataset('gfsanl_4_20180308_0000_000.nc').metpy.parse_cf()
gfs_data = xr.open_dataset('gfs_test_data.nc').metpy.parse_cf()

Could not find variable corresponding to the value of grid_mapping: LatLon_Projection
Found valid latitude/longitude coordinates, assuming latitude_longitude for projection grid_mapping variable
Could not find variable corresponding to the value of grid_mapping: LatLon_Projection
Found valid latitude/longitude coordinates, assuming latitude_longitude for projection grid_mapping variable
Could not find variable corresponding to the value of grid_mapping: LatLon_Projection
Found valid latitude/longitude coordinates, assuming latitude_longitude for projection grid_mapping variable
Could not find variable corresponding to the value of grid_mapping: LatLon_Projection
Found valid latitude/longitude coordinates, assuming latitude_longitude for projection grid_mapping variable
Could not find variable corresponding to the value of grid_mapping: LatLon_Projection
Found valid latitude/longitude coordinates, assuming latitude_longitude for projection grid_mapping variable
Could not find variable c

In [6]:
# Conversion from K to degC

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_tmpc.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_height_above_ground.sel(time=datetime(2018, 3, 8, 0))
data_test = data_var.metpy.convert_units('degC')

print('Conversion from K to degC \n')
error_stats(data_test, out_var)

Conversion from K to degC 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 3.2017650604248047
  Mean Values (GEMPAK): 3.2049618728839624

Max Comparison
  Max Values (MetPy): 35.98681640625
  Max Values (GEMPAK): 35.99

Min Comparison
  Min Values (MetPy): -54.993194580078125
  Min Values (GEMPAK): -54.99

Difference Array
[-0.00319153 -0.00319153 -0.00319153 ... -0.00319458 -0.00319458
 -0.00319458]

Various Statistical Analyses
  Average Absolute Difference: 0.003197004955194289
  RMS Error: 6.270827159720316e-06
  Standard Deviation of Difference: 8.031440975723927e-06
  Max Diff: -0.0031823730468749645
  Min Diff: -0.00321166992187516
  Correlation: 0.999999999999937
  Relative Magnitude Difference: 8.883822673013805e-05



In [7]:
# Conversion from K to degf

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_tmpf.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_height_above_ground.sel(time=datetime(2018, 3, 8, 0))

data_test = data_var.metpy.convert_units('degF')

print('Conversion from K to degf \n')
error_stats(data_test, out_var)

Conversion from K to degf 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 37.763160705566406
  Mean Values (GEMPAK): 37.76291593567251

Max Comparison
  Max Values (MetPy): 96.77625274658203
  Max Values (GEMPAK): 96.78

Min Comparison
  Min Values (MetPy): -66.98776245117188
  Min Values (GEMPAK): -66.99

Difference Array
[0.00224361 0.00224361 0.00224361 ... 0.00223831 0.00223831 0.00223831]

Various Statistical Analyses
  Average Absolute Difference: 0.002442738285147179
  RMS Error: 5.554801255284332e-06
  Standard Deviation of Difference: 0.0028211902449493537
  Max Diff: 0.004252929687503126
  Min Diff: -0.0037942504882835237
  Correlation: 0.9999999975669256
  Relative Magnitude Difference: 2.5241091856942687e-05



In [8]:
# Absolute Value testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_abs_function.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_height_above_ground.sel(time=datetime(2018, 3, 8, 0))

data_test = np.abs((data_var.metpy.convert_units('degC')))

print('Absolute Value \n')
error_stats(data_test, out_var)

Absolute Value 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 19.571880340576172
  Mean Values (GEMPAK): 19.572701254232076

Max Comparison
  Max Values (MetPy): 54.993194580078125
  Max Values (GEMPAK): 54.99

Min Comparison
  Min Values (MetPy): 0.003204345703125
  Min Values (GEMPAK): 0.0

Difference Array
[0.00319153 0.00319153 0.00319153 ... 0.00319458 0.00319458 0.00319458]

Various Statistical Analyses
  Average Absolute Difference: 0.003197004955194289
  RMS Error: 6.270827159720316e-06
  Standard Deviation of Difference: 0.003090161966872017
  Max Diff: 0.00321166992187516
  Min Diff: -0.00321166992187516
  Correlation: 0.9999999647860984
  Relative Magnitude Difference: 5.8134556095645304e-05



In [9]:
# Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = mpcalc.wind_direction(data_var_u, data_var_v)

print('Wind Direction \n')
error_stats(data_test, out_var)

Wind Direction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 214.03001403808594
  Mean Values (GEMPAK): 214.03000611726688

Max Comparison
  Max Values (MetPy): 359.98828125
  Max Values (GEMPAK): 359.99

Min Comparison
  Min Values (MetPy): 0.02294158935546875
  Min Values (GEMPAK): 0.02

Difference Array
[-1.00006104e-03 -1.28173828e-04  7.99560547e-05 ... -5.82275391e-04
 -1.16210938e-03  4.61669922e-03]

Various Statistical Analyses
  Average Absolute Difference: 0.0024985971582893356
  RMS Error: 5.660819761925782e-06
  Standard Deviation of Difference: 0.0028860108199487773
  Max Diff: 0.005029296875022737
  Min Diff: -0.005025634765615905
  Correlation: 0.9999999995326962
  Relative Magnitude Difference: 6.940773598555399e-06



In [10]:
# Cosine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_cosine.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.cos(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Cosine of Wind Direction \n')
error_stats(data_test, out_var)

Cosine of Wind Direction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.0015329133020713925
  Mean Values (GEMPAK): 0.0015334987688519544

Max Comparison
  Max Values (MetPy): 1.0
  Max Values (GEMPAK): 1.0

Min Comparison
  Min Values (MetPy): -1.0
  Min Values (GEMPAK): -1.0

Difference Array
[-1.86295509e-05  4.96351242e-04  4.70650196e-05 ... -4.27130461e-04
  4.97182608e-05  3.11781645e-04]

Various Statistical Analyses
  Average Absolute Difference: 0.00024905513674692225
  RMS Error: 5.648776514298104e-07
  Standard Deviation of Difference: 0.00028798730679233293
  Max Diff: 0.0005002799034118377
  Min Diff: -0.0005008314251899737
  Correlation: 0.9999998920753304
  Relative Magnitude Difference: 0.00024905513674692225



In [11]:
# Sine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_sine.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.sin(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Sine of Wind Direction \n')
error_stats(data_test, out_var)

Sine of Wind Direction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.3287411034107208
  Mean Values (GEMPAK): -0.32874337873191756

Max Comparison
  Max Values (MetPy): 1.0
  Max Values (GEMPAK): 1.0

Min Comparison
  Min Values (MetPy): -1.0
  Min Values (GEMPAK): -1.0

Difference Array
[-1.53689384e-05  2.44963169e-04  4.33412075e-04 ...  4.80057716e-04
  3.99446487e-04  4.48594570e-04]

Various Statistical Analyses
  Average Absolute Difference: 0.00024755109233999485
  RMS Error: 5.631876858860445e-07
  Standard Deviation of Difference: 0.0002871174583032536
  Max Diff: 0.0005004426240921289
  Min Diff: -0.0005006473064422534
  Correlation: 0.9999999188246617
  Relative Magnitude Difference: 0.00024755109233999485



In [12]:
# Tangent of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_tangent.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.tan(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Tangent of Wind Direction \n')
error_stats(data_test, out_var)

Tangent of Wind Direction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -1.277409553527832
  Mean Values (GEMPAK): -1.2726403124038168

Max Comparison
  Max Values (MetPy): 5891.2900390625
  Max Values (GEMPAK): 5891.29

Min Comparison
  Min Values (MetPy): -16253.8408203125
  Min Values (GEMPAK): -16128.84

Difference Array
[-0.00497042 -0.00050993 -0.00116495 ...  0.00294754  0.00186148
 -0.00036686]

Various Statistical Analyses
  Average Absolute Difference: 0.007577882856722329
  RMS Error: 0.0011865922169531178
  Standard Deviation of Difference: 0.6049337959106629
  Max Diff: 27.4277929687496
  Min Diff: -202.2139843749992
  Correlation: 0.9999931147256297
  Relative Magnitude Difference: 1.2862858230500942e-06



In [13]:
# ATAN
gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_arctangent.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.arctan(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Arctangent of Wind Direction \n')
error_stats(data_test, out_var)

Arctangent of Wind Direction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 1.2111256122589111
  Mean Values (GEMPAK): 1.2111266120344724

Max Comparison
  Max Values (MetPy): 1.4129600524902344
  Max Values (GEMPAK): 1.4130000000000003

Min Comparison
  Min Values (MetPy): 0.0004004062502644956
  Min Values (GEMPAK): 0.0

Difference Array
[-3.52842331e-04 -3.91491890e-04 -4.47069168e-04 ... -9.59653854e-05
  4.91332054e-04  6.57548904e-05]

Various Statistical Analyses
  Average Absolute Difference: 0.00024962791333706113
  RMS Error: 5.656045596616701e-07
  Standard Deviation of Difference: 0.0002883567542162603
  Max Diff: 0.0005000200271605415
  Min Diff: -0.0005001134872437696
  Correlation: 0.9999995030024403
  Relative Magnitude Difference: 0.0001766701846220712



In [14]:
# ATAN2 of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_atan2.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)).metpy.quantify()
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)).metpy.quantify()

data_test = np.arctan2(data_var_u, data_var_v)

print('Arctangent2 of Wind Direction \n')
error_stats(data_test, out_var)

Arctangent2 of Wind Direction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.5939357280731201
  Mean Values (GEMPAK): 0.593936026469683

Max Comparison
  Max Values (MetPy): 3.141387939453125
  Max Values (GEMPAK): 3.141

Min Comparison
  Min Values (MetPy): -3.1411921977996826
  Min Values (GEMPAK): -3.141

Difference Array
[ 6.23350143e-05 -2.14061737e-05 -1.16472244e-04 ... -3.32740784e-04
 -6.99472427e-05 -4.47378159e-05]

Various Statistical Analyses
  Average Absolute Difference: 0.0002500652357785991
  RMS Error: 5.663817823264338e-07
  Standard Deviation of Difference: 0.00028875462864006536
  Max Diff: 0.0005000801086425533
  Min Diff: -0.0005001392364500923
  Correlation: 0.9999999846429966
  Relative Magnitude Difference: 7.96034238999887e-05



In [15]:
# Arcsine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_arcsine.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat.metpy.quantify()
data_var_lon = gfs_data.lon.metpy.quantify()

xx, yy = np.meshgrid(data_var_lon, data_var_lat)

data_test = np.arcsin((np.deg2rad(yy) * units.radian)/(np.pi / 2))

print('Arcsine of Wind Direction \n')
error_stats(data_test, out_var)

Arcsine of Wind Direction 

Is Metpy Unit Array Quantity? True

Mean Comparison
  Mean Values (MetPy): 4.8843155298072816e-08
  Mean Values (GEMPAK): 6.998266403300464e-18

Max Comparison
  Max Values (MetPy): 1.570796251296997
  Max Values (GEMPAK): 1.5710000000000002

Min Comparison
  Min Values (MetPy): -1.570796251296997
  Min Values (GEMPAK): -1.5710000000000002

Difference Array
[-0.00020375 -0.00020375 -0.00020375 ...  0.00020375  0.00020375
  0.00020375]

Various Statistical Analyses
  Average Absolute Difference: 0.00025933956115447085
  RMS Error: 5.919930774197232e-07
  Standard Deviation of Difference: 0.00030181198203691373
  Max Diff: 0.0004989566802979795
  Min Diff: -0.0004989566802979795
  Correlation: 0.9999999043620693
  Relative Magnitude Difference: 0.00016510070032337785



In [16]:
# Arccosine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_arccosine.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

xx, yy = np.meshgrid(data_var_lon, data_var_lat)

data_test = np.arccos((np.deg2rad(yy) * units.radian)/(np.pi / 2))

print('Arccosine of Wind Direction \n')
error_stats(data_test, out_var)

Arccosine of Wind Direction 

Is Metpy Unit Array Quantity? True

Mean Comparison
  Mean Values (MetPy): 1.570796251296997
  Mean Values (GEMPAK): 1.570770083102493

Max Comparison
  Max Values (MetPy): 3.141592502593994
  Max Values (GEMPAK): 3.1420000000000003

Min Comparison
  Min Values (MetPy): 0.0
  Min Values (GEMPAK): 0.0

Difference Array
[ 0.         0.         0.        ... -0.0004075 -0.0004075 -0.0004075]

Various Statistical Analyses
  Average Absolute Difference: 0.00025326071254434866
  RMS Error: 5.658199340957311e-07
  Standard Deviation of Difference: 0.0002872721130548563
  Max Diff: 0.0004998493194580123
  Min Diff: -0.0004988970756532041
  Correlation: 0.9999999128255722
  Relative Magnitude Difference: 8.061539245947169e-05



In [17]:
# Exponential testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_exponential.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var ** 1.1

print('Exponential \n')
error_stats(data_test, out_var)

Exponential 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 436.7687683105469
  Mean Values (GEMPAK): 436.7687962792398

Max Comparison
  Max Values (MetPy): 484.9851379394531
  Max Values (GEMPAK): 484.98519999999996

Min Comparison
  Min Values (MetPy): 385.62945556640625
  Min Values (GEMPAK): 385.6295

Difference Array
[-3.50585938e-05 -3.50585938e-05 -3.50585938e-05 ...  4.54101564e-06
  4.54101564e-06  4.54101564e-06]

Various Statistical Analyses
  Average Absolute Difference: 3.0305831406568193e-05
  RMS Error: 7.303867932000304e-08
  Standard Deviation of Difference: 3.554043409065714e-05
  Max Diff: 6.0839843683879735e-05
  Min Diff: -0.00012133789061863354
  Correlation: 0.9999999999991587
  Relative Magnitude Difference: 6.248816517414943e-08



In [18]:
# Integer Exponential testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_integer_exponential.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var ** np.rint(1.6)

print('Integer Exponential \n')
error_stats(data_test, out_var)

# NOTES: GEMPAK NINT(2.5) = 3; numpy.round(2.5) = 2.0 (as numpy rounds to nearest EVEN value); numpy.rint(2.5) = 2.0

Integer Exponential 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 63345.71484375
  Mean Values (GEMPAK): 63345.71344536781

Max Comparison
  Max Values (MetPy): 76408.6484375
  Max Values (GEMPAK): 76408.65

Min Comparison
  Min Values (MetPy): 50364.84765625
  Min Values (GEMPAK): 50364.85

Difference Array
[-0.0034375  -0.0034375  -0.0034375  ...  0.00265625  0.00265625
  0.00265625]

Various Statistical Analyses
  Average Absolute Difference: 0.004406293883298373
  RMS Error: 1.2258116504030456e-05
  Standard Deviation of Difference: 0.005843136573907865
  Max Diff: 0.005000000004656613
  Min Diff: -0.026562500002910383
  Correlation: 0.9999999999996713
  Relative Magnitude Difference: 5.76674757819148e-08



In [19]:
# Conversion integer of degf

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_integer_tmpf.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degF').astype('int')

print('Conversion integer of degf \n')
error_stats(data_test, out_var)

Conversion integer of degf 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -7.333387196060326
  Mean Values (GEMPAK): -7.333387196060326

Max Comparison
  Max Values (MetPy): 37
  Max Values (GEMPAK): 37.0

Min Comparison
  Min Values (MetPy): -55
  Min Values (GEMPAK): -55.0

Difference Array
[0. 0. 0. ... 0. 0. 0.]

Various Statistical Analyses
  Average Absolute Difference: 0.0
  RMS Error: 0.0
  Standard Deviation of Difference: 0.0
  Max Diff: 0.0
  Min Diff: 0.0
  Correlation: 1.0
  Relative Magnitude Difference: 0.0



In [20]:
# Natural Log of Geopotential Height

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_natural_log.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.log(data_var/units.meter)

print('Natural Log of Geopotential Height \n')
error_stats(data_test, out_var)

Natural Log of Geopotential Height 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 8.608513832092285
  Mean Values (GEMPAK): 8.60851575100031

Max Comparison
  Max Values (MetPy): 8.68575668334961
  Max Values (GEMPAK): 8.686

Min Comparison
  Min Values (MetPy): 8.463985443115234
  Min Values (GEMPAK): 8.464

Difference Array
[ 0.00027254  0.00027254  0.00027254 ... -0.0003763  -0.0003763
 -0.0003763 ]

Various Statistical Analyses
  Average Absolute Difference: 0.00025065645785580175
  RMS Error: 5.667651015090695e-07
  Standard Deviation of Difference: 0.0002889464771773479
  Max Diff: 0.0005009613037092464
  Min Diff: -0.0005001983642571872
  Correlation: 0.9999899073180549
  Relative Magnitude Difference: 2.8858332899919274e-05



In [21]:
# Log Base 10 of Geopotential Height

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_log_base10.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.log10(data_var/units.meter)

print('Log Base 10 of Geopotential Height \n')
error_stats(data_test, out_var)

Log Base 10 of Geopotential Height 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 3.7386300563812256
  Mean Values (GEMPAK): 3.7386270660203142

Max Comparison
  Max Values (MetPy): 3.7721760272979736
  Max Values (GEMPAK): 3.7720000000000002

Min Comparison
  Min Values (MetPy): 3.6758623123168945
  Min Values (GEMPAK): 3.676

Difference Array
[-9.81693268e-05 -9.81693268e-05 -9.81693268e-05 ... -2.25870132e-04
 -2.25870132e-04 -2.25870132e-04]

Various Statistical Analyses
  Average Absolute Difference: 0.0002501943947315712
  RMS Error: 5.665074380795213e-07
  Standard Deviation of Difference: 0.00028880132233336756
  Max Diff: 0.0005000667572017115
  Min Diff: -0.0005000190734865129
  Correlation: 0.9999465450374277
  Relative Magnitude Difference: 6.632627770310776e-05



In [22]:
# Conversion round integer of degC

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_round_integer.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.rint(data_var.metpy.convert_units('degC'))

print('Conversion round integer of degC \n')
error_stats(data_test, out_var)

Conversion round integer of degC 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -21.889684677124023
  Mean Values (GEMPAK): -21.889685287780857

Max Comparison
  Max Values (MetPy): 3.0
  Max Values (GEMPAK): 3.0

Min Comparison
  Min Values (MetPy): -49.0
  Min Values (GEMPAK): -49.0

Difference Array
[0. 0. 0. ... 0. 0. 0.]

Various Statistical Analyses
  Average Absolute Difference: 0.0
  RMS Error: 0.0
  Standard Deviation of Difference: 0.0
  Max Diff: 0.0
  Min Diff: 0.0
  Correlation: 1.0
  Relative Magnitude Difference: 0.0



In [23]:
# Square Root

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_square_root.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.sqrt(data_var)

print('Square Root \n')
error_stats(data_test, out_var)

Square Root 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 74.05231475830078
  Mean Values (GEMPAK): 74.05232495498615

Max Comparison
  Max Values (MetPy): 76.92864227294922
  Max Values (GEMPAK): 76.9286

Min Comparison
  Min Values (MetPy): 68.85431671142578
  Min Values (GEMPAK): 68.85430000000001

Difference Array
[-3.72314453e-06 -3.72314453e-06 -3.72314453e-06 ... -1.11816406e-05
 -1.11816406e-05 -1.11816406e-05]

Various Statistical Analyses
  Average Absolute Difference: 2.4943109748916872e-05
  RMS Error: 5.668008634892868e-08
  Standard Deviation of Difference: 2.8885914350421697e-05
  Max Diff: 5.976562499654392e-05
  Min Diff: -5.2441406253933565e-05
  Correlation: 0.999999999925485
  Relative Magnitude Difference: 3.24236968337705e-07



In [24]:
# Addition

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_add.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var + 5 * units.delta_degC

print('Addition \n')
error_stats(data_test, out_var)

Addition 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 256.2801513671875
  Mean Values (GEMPAK): 256.28018177554634

Max Comparison
  Max Values (MetPy): 281.421142578125
  Max Values (GEMPAK): 281.4211

Min Comparison
  Min Values (MetPy): 229.421142578125
  Min Values (GEMPAK): 229.42110000000002

Difference Array
[-5.43701172e-05 -5.43701172e-05 -5.43701172e-05 ...  3.95263672e-05
  3.95263672e-05  3.95263672e-05]

Various Statistical Analyses
  Average Absolute Difference: 4.943907657491617e-05
  RMS Error: 9.899529510770071e-08
  Standard Deviation of Difference: 4.731823461283548e-05
  Max Diff: 4.257812497598934e-05
  Min Diff: -6.962890626027729e-05
  Correlation: 0.9999999999945128
  Relative Magnitude Difference: 1.7567648301758795e-07



In [25]:
# Subtraction

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_subtract.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var - 5 * units.degC

print('Subtraction \n')
error_stats(data_test, out_var)

Subtraction 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -26.869827270507812
  Mean Values (GEMPAK): 246.28018181248078

Max Comparison
  Max Values (MetPy): -1.728851318359375
  Max Values (GEMPAK): 271.4211

Min Comparison
  Min Values (MetPy): -53.728851318359375
  Min Values (GEMPAK): 219.42110000000002

Difference Array
[-273.15004827 -273.15004827 -273.15004827 ... -273.14995437 -273.14995437
 -273.14995437]

Various Statistical Analyses
  Average Absolute Difference: 273.15001148393486
  RMS Error: 0.5357736787131016
  Standard Deviation of Difference: 4.755216397719556e-05
  Max Diff: -273.1499513183594
  Min Diff: -273.15006352539064
  Correlation: 0.9999999999944589
  Relative Magnitude Difference: -157.99508528191166



In [26]:
# Multiplication

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_multiplication.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var * 2

print('Multiplication \n')
error_stats(data_test, out_var)

Multiplication 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 502.5603332519531
  Mean Values (GEMPAK): 502.5603432440752

Max Comparison
  Max Values (MetPy): 552.84228515625
  Max Values (GEMPAK): 552.8423

Min Comparison
  Min Values (MetPy): 448.84228515625
  Min Values (GEMPAK): 448.8423

Difference Array
[-8.74023442e-06 -8.74023442e-06 -8.74023442e-06 ... -2.09472657e-05
 -2.09472657e-05 -2.09472657e-05]

Various Statistical Analyses
  Average Absolute Difference: 1.6571400395329476e-05
  RMS Error: 3.905862349187846e-08
  Standard Deviation of Difference: 1.3329098428728595e-05
  Max Diff: 9.570312499818101e-06
  Min Diff: -3.9257812545656634e-05
  Correlation: 0.9999999999998914
  Relative Magnitude Difference: 2.9974914799879855e-08



In [27]:
# Quotient

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_quotient.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var / 2

print('Division \n')
error_stats(data_test, out_var)

Division 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 125.64008331298828
  Mean Values (GEMPAK): 125.64011081101877

Max Comparison
  Max Values (MetPy): 138.2105712890625
  Max Values (GEMPAK): 138.2106

Min Comparison
  Min Values (MetPy): 112.2105712890625
  Min Values (GEMPAK): 112.2106

Difference Array
[-2.71850586e-05 -2.71850586e-05 -2.71850586e-05 ... -3.02368164e-05
 -3.02368164e-05 -3.02368164e-05]

Various Statistical Analyses
  Average Absolute Difference: 2.8698503627788848e-05
  RMS Error: 5.6669252172195926e-08
  Standard Deviation of Difference: 3.332274606180096e-06
  Max Diff: -2.260742186876996e-05
  Min Diff: -3.4814453130138645e-05
  Correlation: 0.9999999999998918
  Relative Magnitude Difference: 2.0764333263457068e-07



In [28]:
# Less Than

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_less_than.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test < -10 * units.degC

print('Less Than -10 C \n')
#error_stats(data_test.where(data_mask).values.flatten(), out_var[data_mask])
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

Less Than -10 C 

-48.728851318359375 degree_Celsius
-48.7289
-10.028839111328125 degree_Celsius
-10.0288


In [29]:
# Greater Than

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_greater_than.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test > -10 * units.degC

print('Greater Than -10 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

Greater Than -10 C 

-9.928863525390625 degree_Celsius
-9.9288
3.271148681640625 degree_Celsius
3.2711


In [30]:
# Less Than Equal To

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_less_than_equal.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test <= -10.5 * units.degC

print('Less Than or Equal -10.5 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

Less Than or Equal -10.5 C 

-48.728851318359375 degree_Celsius
-48.7289
-10.528839111328125 degree_Celsius
-10.528800000000002


In [31]:
# Greater Than

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_greater_than_equal.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test >= -10.5 * units.degC

print('Greater Than or Equal -10.5 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

Greater Than or Equal -10.5 C 

-10.428863525390625 degree_Celsius
-10.4288
3.271148681640625 degree_Celsius
3.2711


In [32]:
# Boolean Between

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_boolean_between.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = (data_test > -25 * units.degC) & (data_test < -10 * units.degC)

print('Boolean between -10 C and -25 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

Boolean between -10 C and -25 C 

-24.928848266601562 degree_Celsius
-24.928800000000003
-10.028839111328125 degree_Celsius
-10.0288


In [33]:
# Advection

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_advection.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_degC = data_var.metpy.convert_units('degC')

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

adv = mpcalc.advection(data_var_degC.data, [data_var_u.data, data_var_v.data], deltas=[dx, dy])

print('Advection \n')
error_stats(adv.to('degC/s')[2:-2,2:-2]*1e6, out_var[2:-2,2:-2]*1e6)

Advection 

Is Metpy Unit Array Quantity? True

Mean Comparison
  Mean Values (MetPy): 2.0404344209098246
  Mean Values (GEMPAK): 2.040368503826111

Max Comparison
  Max Values (MetPy): 2320.5779287453333
  Max Values (GEMPAK): 2320.4939999999997

Min Comparison
  Min Values (MetPy): -2019.5386162161042
  Min Values (GEMPAK): -2019.4610000000002

Difference Array
[ 7.02056488e-04  6.01604503e-04  5.21242915e-04 ...  2.87552313e-04
  3.86360409e-05 -2.93426532e-04]

Various Statistical Analyses
  Average Absolute Difference: 0.00235445940012441
  RMS Error: 8.220368336526208e-06
  Standard Deviation of Difference: 0.00415553807344803
  Max Diff: 0.11320173921410515
  Min Diff: -0.09751233981415908
  Correlation: 0.9999999993595247
  Relative Magnitude Difference: 1.014600445414645e-06



/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [34]:
# Average

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_average.out')

out_var = gempak_out_var * 10**scale

data_var_500 = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_850 = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_test = xr.concat([data_var_500, data_var_850], dim='level').mean(axis=0)

print('Average \n')
error_stats(data_test, out_var)

Average 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 261.8683166503906
  Mean Values (GEMPAK): 261.8683583487227

Max Comparison
  Max Values (MetPy): 287.3605651855469
  Max Values (GEMPAK): 287.36060000000003

Min Comparison
  Min Values (MetPy): 231.61056518554688
  Min Values (GEMPAK): 231.6106

Difference Array
[-1.65039063e-05 -1.65039063e-05 -1.65039063e-05 ... -2.26074219e-05
 -2.26074219e-05 -2.26074219e-05]

Various Statistical Analyses
  Average Absolute Difference: 2.866707261079589e-05
  RMS Error: 5.988206923635636e-08
  Standard Deviation of Difference: 1.0499345461375007e-05
  Max Diff: -4.2968750335603545e-06
  Min Diff: -5.312500002219167e-05
  Correlation: 0.9999999999997726
  Relative Magnitude Difference: 9.975993954593507e-08



In [35]:
# Corilois Parameter

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_coriolis.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat

data_test = mpcalc.coriolis_parameter(data_var_lat)

print('Coriolis Parameter \n')
error_stats(data_test[:180], out_var[:180, 0])


Coriolis Parameter 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 9.3250630015973e-05
  Mean Values (GEMPAK): 9.325039999999999e-05

Max Comparison
  Max Values (MetPy): 0.0001458423212170601
  Max Values (GEMPAK): 0.000145842

Min Comparison
  Min Values (MetPy): 1.272698227694491e-06
  Min Values (GEMPAK): 1.273e-06

Difference Array
[ 3.21217060e-10  7.62385443e-10  1.14994422e-10  3.49940166e-10
  4.81774591e-10  5.10497697e-10  4.50661398e-10  2.87713781e-10
  6.53105900e-11  7.39796080e-10  3.54825996e-10 -1.04151577e-10
  3.91967192e-10 -1.71369612e-10  2.34941840e-10  6.10901549e-10
 -1.43866539e-11  3.73629145e-10  7.60397032e-10  1.89572752e-10
  6.46604389e-10  1.75147690e-10  7.89754570e-10  4.90425028e-10
  3.20814811e-10  2.80923918e-10  3.85304265e-10  6.77611597e-10
  1.86949745e-10 -1.01233207e-10 -1.43281512e-10  1.04460575e-10
  6.41993053e-10  4.98419754e-10  7.02844508e-10  2.84371145e-10
  2.86655411e-10  7.24249221e-10  6.26256406e-10  2.

/Users/kgoebber/unidata_stuff/MetPy/src/metpy/calc/basic.py:1183: UserWarning: Input over 1.5707963267948966 radians. Ensure proper units are given.
  warnings.warn('Input over {} radians. '


In [36]:
# Absolute Vorticity

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_absolute_vorticity.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

avor = mpcalc.absolute_vorticity(data_var_u, data_var_v, latitude=data_var_lat.values[:, None] * units.degree)

print('Absolute Vorticity \n')
error_stats(avor[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

avor_corrected = avor + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat.values[:, None]))

print('\nAbsolute Vorticity with spherical corrections \n')
error_stats(avor_corrected[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

/Users/kgoebber/unidata_stuff/MetPy/src/metpy/xarray.py:325: UserWarning: More than one vertical coordinate present for variable "u-component_of_wind_isobaric".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


Absolute Vorticity 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.004190604253050569
  Mean Values (GEMPAK): -0.04004442853197606

Max Comparison
  Max Values (MetPy): 49.73547343789765
  Max Values (GEMPAK): 50.118100000000005

Min Comparison
  Min Values (MetPy): -113.62620399669227
  Min Values (GEMPAK): -113.51350000000001

Difference Array
[4.85732109 4.81689144 4.77942092 ... 4.10390334 4.11446557 4.12498071]

Various Statistical Analyses
  Average Absolute Difference: 0.28708526551012487
  RMS Error: 0.0010811457014453193
  Standard Deviation of Difference: 0.5425110769645497
  Max Diff: 6.099997675373704
  Min Diff: -7.053020063458463
  Correlation: 0.9989036000741114
  Relative Magnitude Difference: 0.005772243545014098


Absolute Vorticity with spherical corrections 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.04000841071717019
  Mean Values (GEMPAK): -0.04004442853197606

Max Comparison
  Max Values (MetPy): 49.9986221

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [37]:
# Vorticity

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_vorticity.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat.values

vor = mpcalc.vorticity(data_var_u, data_var_v)

print('Vorticity \n')
error_stats(vor[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

vor_corrected = vor + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None]))

print('\nVorticity with spherical corrections \n')
error_stats(vor_corrected[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

/Users/kgoebber/unidata_stuff/MetPy/src/metpy/xarray.py:325: UserWarning: More than one vertical coordinate present for variable "u-component_of_wind_isobaric".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


Vorticity 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.004190604253050841
  Mean Values (GEMPAK): -0.04004379610999328

Max Comparison
  Max Values (MetPy): 36.85836141622791
  Max Values (GEMPAK): 37.241

Min Comparison
  Min Values (MetPy): -109.36218724456099
  Min Values (GEMPAK): -109.249

Difference Array
[4.8571861  4.81715645 4.77918593 ... 4.10353833 4.11470056 4.12481571]

Various Statistical Analyses
  Average Absolute Difference: 0.2870962188320009
  RMS Error: 0.0010811573629413067
  Standard Deviation of Difference: 0.5425170190877117
  Max Diff: 6.099662681357114
  Min Diff: -7.052885069441876
  Correlation: 0.9917837585441162
  Relative Magnitude Difference: 0.007789174770683076


Vorticity with spherical corrections 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.04000841071717095
  Mean Values (GEMPAK): -0.04004379610999328

Max Comparison
  Max Values (MetPy): 37.121510088502745
  Max Values (GEMPAK): 37.241

Mi

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [67]:
# Brunt Vaisala Frequency
gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_brunt_vaisala_freq.out')

out_var = gempak_out_var * 10**scale

data_var_p = gfs_data.isobaric2.metpy.sel(vertical=slice(55000, 45000))
data_var_hght = gfs_data.Geopotential_height_isobaric.metpy.sel(time=datetime(2018, 3, 8, 0), vertical=slice(55000, 45000))
data_var_tmpk = gfs_data.Temperature_isobaric.metpy.sel(time=datetime(2018, 3, 8, 0), vertical=slice(55000, 45000))

data_var_p_3D, _ = xr.broadcast(data_var_p, data_var_hght)

data_var_thta = mpcalc.potential_temperature(data_var_p, data_var_tmpk)

# layer = mpcalc.mixed_layer((data_var_p.data[::-1, None, None] * units.Pa).to('hPa'), data_var_thta.metpy.quantify(),
#                         height=data_var_hght.metpy.quantify(), bottom=850*units.hPa, depth=50*units.hPa)
# height_layer = data_var_hght.metpy.sel(vertical=500*units.hPa)-data_var_hght.metpy.sel(vertical=850*units.hPa)
data_test = mpcalc.brunt_vaisala_frequency_squared(data_var_hght.metpy.quantify().data, data_var_thta.data)
data_test1 = np.average(data_test, axis=0)
data_test2 = mpcalc.mixed_layer(data_var_p.values * units.Pa, data_test[:, 100, 100])

print('Brunt-Vaisala Frequency Squared \n')
#print(data_test2, out_var[100, 100])
#print(np.average(data_test[:, 100, 100]), out_var[100, 100])
error_stats(data_test1, out_var)

Brunt-Vaisala Frequency Squared 

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): 0.0001389994431519881
  Mean Values (GEMPAK): 0.00013894005149661437

Max Comparison
  Max Values (MetPy): 0.00038105191197246313
  Max Values (GEMPAK): 0.00037783099999999997

Min Comparison
  Min Values (MetPy): 7.671737876080442e-06
  Min Values (GEMPAK): 7.701999999999998e-06

Difference Array
[-5.98838433e-07 -5.98838433e-07 -5.98838433e-07 ...  1.20470553e-06
  1.20470553e-06  1.20470553e-06]

Various Statistical Analyses
  Average Absolute Difference: 5.708693625996961e-07
  RMS Error: 1.6746154682398257e-09
  Standard Deviation of Difference: 8.516893957752941e-07
  Max Diff: 1.066407989804447e-05
  Min Diff: -1.081401802806558e-05
  Correlation: 0.9998125197108438
  Relative Magnitude Difference: 0.0014981406592206004



In [68]:
# Cross Product

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_cross_product.out')

out_var = gempak_out_var * 10**scale


data_var_u_500 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v_500 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_u_850 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_v_850 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_test = np.cross(xr.concat([data_var_u_850, data_var_v_850], dim='component'),
                     xr.concat([data_var_u_500, data_var_v_500], dim='component'), axis=0)

print('Cross Product of a Vector \n')
error_stats(data_test, out_var)

Cross Product of a Vector 

Is Xarray with Quantity? False
Is Metpy Unit Array? False
I got back just a numpy array

Mean Comparison
  Mean Values (MetPy): 0.6471587419509888
  Mean Values (GEMPAK): 0.6471622037550014

Max Comparison
  Max Values (MetPy): 898.768310546875
  Max Values (GEMPAK): 898.77

Min Comparison
  Min Values (MetPy): -1079.914306640625
  Min Values (GEMPAK): -1079.91

Difference Array
[-0.00246536  0.00195541 -0.00324356 ... -0.00256104  0.00397491
 -0.00079559]

Various Statistical Analyses
  Average Absolute Difference: 0.0025036061774914497
  RMS Error: 5.6676532305803615e-06
  Standard Deviation of Difference: 0.002889501206684315
  Max Diff: 0.00510742187498181
  Min Diff: -0.005034179687498863
  Correlation: 0.9999999993730183
  Relative Magnitude Difference: 2.7855968530622497e-06



In [69]:
# Density

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_density.out')

out_var = gempak_out_var * 10**scale

data_var_850 = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_test = mpcalc.density(850*units.hPa, data_var_850, mixing_ratio=0)

print('Density of Dry Air \n')
error_stats(data_test, out_var)

Density of Dry Air 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 1.091292142868042
  Mean Values (GEMPAK): 1.0913651738996615

Max Comparison
  Max Values (MetPy): 1.2675831317901611
  Max Values (GEMPAK): 1.268

Min Comparison
  Min Values (MetPy): 0.9727576375007629
  Min Values (GEMPAK): 0.9730000000000001

Difference Array
[0.00024106 0.00024106 0.00024106 ... 0.00035662 0.00035662 0.00035662]

Various Statistical Analyses
  Average Absolute Difference: 0.0002548572225455517
  RMS Error: 5.828888206459952e-07
  Standard Deviation of Difference: 0.00028807701839174025
  Max Diff: 0.0004349431991577113
  Min Diff: -0.0005749835968018591
  Correlation: 0.9999918018085193
  Relative Magnitude Difference: 0.00020105760021090388



In [70]:
# Partial Derivative in X

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_partial_x_derivative.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

# Gives a unit_array, not a DataArray
data_test = mpcalc.first_derivative(data_var, delta=dx, axis=1)

print('Partial Derivative x-direction \n')
error_stats(data_test[3:-3,3:-3], out_var[3:-3,3:-3])

Partial Derivative x-direction 

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): 4.6522560884050245e-09
  Mean Values (GEMPAK): 4.651572178167039e-09

Max Comparison
  Max Values (MetPy): 0.0002065521189220411
  Max Values (GEMPAK): 0.00020654499999999998

Min Comparison
  Min Values (MetPy): -0.00014758836234407893
  Min Values (GEMPAK): -0.00014758299999999999

Difference Array
[0. 0. 0. ... 0. 0. 0.]

Various Statistical Analyses
  Average Absolute Difference: 2.9880503747579907e-10
  RMS Error: 9.194843784780717e-13
  Standard Deviation of Difference: 4.62921299591354e-10
  Max Diff: 7.118922041107496e-09
  Min Diff: -6.701323303738506e-09
  Correlation: 0.999999998309515
  Relative Magnitude Difference: 1.4466326418494743e-06



/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [71]:
# Partial Derivative in Y

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_partial_y_derivative.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

data_test = mpcalc.first_derivative(data_var, delta=dy, axis=0)

print('Partial Derivative y-direction \n')
error_stats(data_test, out_var)

Partial Derivative y-direction 

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): -2.6868207403016607e-07
  Mean Values (GEMPAK): -2.6944623345644844e-07

Max Comparison
  Max Values (MetPy): 6.385191897729202e-05
  Max Values (GEMPAK): 6.385e-05

Min Comparison
  Min Values (MetPy): -5.9355308830318174e-05
  Min Values (GEMPAK): -5.935299999999999e-05

Difference Array
[ 1.79861661e-06  1.79861661e-06  1.79861661e-06 ... -2.69920089e-06
 -4.49715967e-06 -4.49715967e-06]

Various Statistical Analyses
  Average Absolute Difference: 6.957906429492021e-09
  RMS Error: 2.2513416862549713e-10
  Standard Deviation of Difference: 1.1477614577237291e-07
  Max Diff: 3.5975778135480963e-06
  Min Diff: -4.497159669907485e-06
  Correlation: 0.9998936057724886
  Relative Magnitude Difference: 0.00010896941769230924



In [72]:
# Deformation (shearing, stretching, total)

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_total_deformation.out')
out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_shear_deformation.out')
out_var_shr = gempak_out_var * 10**scale

# Shearing Deformation
print()
shr_def = mpcalc.shearing_deformation(data_var_u, data_var_v)
print('Shearing Deformation \n')
error_stats(shr_def[3:-3,3:-3]*1e5, out_var_shr[3:-3,3:-3]*1e5)

print()
shr_def_corrected = (mpcalc.shearing_deformation(data_var_u, data_var_v)
                     + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
print('Shearing Deformation with spherical corrections \n')
error_stats(shr_def_corrected[3:-3,3:-3]*1e5, out_var_shr[3:-3,3:-3]*1e5)

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_stretch_deformation.out')
out_var_str = gempak_out_var * 10**scale

# Streching Deformation
print()
str_def = mpcalc.stretching_deformation(data_var_u, data_var_v)
print('Stretching Deformation \n')
error_stats(str_def[3:-3,3:-3]*1e5, out_var_str[3:-3,3:-3]*1e5)

print()
str_def_corrected = (mpcalc.stretching_deformation(data_var_u, data_var_v)
                     - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
print('Stretching Deformation with spherical corrections \n')
error_stats(str_def_corrected[3:-3,3:-3]*1e5, out_var_str[3:-3,3:-3]*1e5)

deformation = mpcalc.total_deformation(data_var_u, data_var_v)

print()
# Total Deformation
print('Total Deformation \n')
error_stats(deformation[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)



print()
deformation_corrected = np.sqrt(str_def_corrected**2 + shr_def_corrected**2)
print('Total Deformation with spherical corrections \n')
error_stats(deformation_corrected[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)


Shearing Deformation 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.01956842862185442
  Mean Values (GEMPAK): -0.06379221209610605

Max Comparison
  Max Values (MetPy): 40.57647801116041
  Max Values (GEMPAK): 40.5062

Min Comparison
  Min Values (MetPy): -58.224550970477154
  Min Values (GEMPAK): -58.1335

Difference Array
[4.85578467 4.81535536 4.77788484 ... 4.11869633 4.12897675 4.13940426]

Various Statistical Analyses
  Average Absolute Difference: 0.28553918087267277
  RMS Error: 0.0010795050840958313
  Standard Deviation of Difference: 0.5416832681361613
  Max Diff: 6.094374613415837
  Min Diff: -7.065571001002665
  Correlation: 0.9856125001435396
  Relative Magnitude Difference: 0.007037061737939312


Shearing Deformation with spherical corrections 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.06376744359207616
  Mean Values (GEMPAK): -0.06379221209610605

Max Comparison
  Max Values (MetPy): 40.58706141589078
  Max Val

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


Stretching Deformation 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.003017691738283728
  Mean Values (GEMPAK): 0.0011109953840691203

Max Comparison
  Max Values (MetPy): 37.44665984360355
  Max Values (GEMPAK): 37.3033

Min Comparison
  Min Values (MetPy): -43.393966319113105
  Min Values (GEMPAK): -43.369899999999994

Difference Array
[2.74199991 2.77780644 2.8137124  ... 0.53647268 0.50633196 0.46879833]

Various Statistical Analyses
  Average Absolute Difference: 0.2180508883893273
  RMS Error: 0.0009668908897925905
  Standard Deviation of Difference: 0.48677148516374025
  Max Diff: 5.707644731085825
  Min Diff: -6.794653928931952
  Correlation: 0.9885697627122795
  Relative Magnitude Difference: 0.005822972978097902


Stretching Deformation with spherical corrections 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.0011290844510445054
  Mean Values (GEMPAK): 0.0011109953840691203

Max Comparison
  Max Values (MetPy): 37.432257

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [73]:
# Divergence

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_divergence.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

divergence = mpcalc.divergence(data_var_u, data_var_v)

print('Divergence \n')
error_stats(divergence[3:-3,3:-3]*1e4, out_var[3:-3,3:-3]*1e4)

divergence_corrected = (divergence - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))

print('\nDivergence with spherical corrections \n')
error_stats(divergence_corrected[3:-3,3:-3]*1e4, out_var[3:-3,3:-3]*1e4)

Divergence 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 9.001892081182317e-08
  Mean Values (GEMPAK): 0.0004153544403676964

Max Comparison
  Max Values (MetPy): 2.1238656985165267
  Max Values (GEMPAK): 1.95621

Min Comparison
  Min Values (MetPy): -2.274842803371438
  Min Values (GEMPAK): -2.5082699999999996

Difference Array
[0.27540709 0.27900643 0.28260584 ... 0.05450979 0.05152386 0.04777986]

Various Statistical Analyses
  Average Absolute Difference: 0.021879332232986246
  RMS Error: 9.679631608226306e-05
  Standard Deviation of Difference: 0.04873111444708866
  Max Diff: 0.571507905325638
  Min Diff: -0.6807457620853754
  Correlation: 0.9522377411853302
  Relative Magnitude Difference: 0.010301655254505252


Divergence with spherical corrections 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.0004147676378536353
  Mean Values (GEMPAK): 0.0004153544403676964

Max Comparison
  Max Values (MetPy): 1.950768197106774
  Max Values

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [74]:
# Dot Product

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dot_product.out')

out_var = gempak_out_var * 10**scale


data_var_u_500 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v_500 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_u_850 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_v_850 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

#data_test = data_var_u_850 * data_var_u_500 + data_var_v_850 * data_var_v_500
data_test2 = xr.dot(xr.concat([data_var_u_850, data_var_v_850], dim='component'),
                    xr.concat([data_var_u_500, data_var_v_500], dim='component'),
                    dims='component')

print('Dot Product of a Vector \n')
error_stats(data_test2, out_var)

Dot Product of a Vector 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 103.11414337158203
  Mean Values (GEMPAK): 103.11412138350265

Max Comparison
  Max Values (MetPy): 1736.34765625
  Max Values (GEMPAK): 1736.35

Min Comparison
  Min Values (MetPy): -332.6321716308594
  Min Values (GEMPAK): -332.63

Difference Array
[ 0.0038303   0.00292934  0.00176083 ... -0.00101254  0.00370667
 -0.0025692 ]

Various Statistical Analyses
  Average Absolute Difference: 0.0024981698796061698
  RMS Error: 5.658121888140804e-06
  Standard Deviation of Difference: 0.0028845972228634907
  Max Diff: 0.005161132812531832
  Min Diff: -0.005013427734368747
  Correlation: 0.999999999885552
  Relative Magnitude Difference: 1.4387498209900438e-06



In [75]:
# Frotnogenesis

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_frontogenesis.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_tmpk = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

thta = mpcalc.potential_temperature(850*units.hPa, data_var_tmpk)

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

fronto = mpcalc.frontogenesis(thta, data_var_u, data_var_v) * 1000 * 100 * 3600 * 3

# Frontogenesis undefined at a couple of points, ignore them for determining differences
#ignore = (out_var != -9999.0) * ~np.isnan(fronto)

print('Frontogenesis \n')
error_stats(fronto, out_var)

# Get gradients of potential temperature in both x and y
ddy_thta = mpcalc.first_derivative(thta, delta=dy, axis=-2)
ddx_thta = mpcalc.first_derivative(thta, delta=dx, axis=-1)

# Compute the magnitude of the potential temperature gradient
mag_thta = np.sqrt(ddx_thta**2 + ddy_thta**2)

# Get the shearing, stretching, and total deformation of the wind field
shrd = (mpcalc.shearing_deformation(data_var_u, data_var_v) 
        + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
strd = (mpcalc.stretching_deformation(data_var_u, data_var_v)
        - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
tdef = np.sqrt(shrd**2 + strd**2)

# Get the divergence of the wind field
div = (mpcalc.divergence(data_var_u, data_var_v)
       - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))

# Compute the angle (beta) between the wind field and the gradient of potential temperature
psi = 0.5 * np.arctan2(shrd, strd)
beta = np.arcsin((-ddx_thta * np.cos(psi) - ddy_thta * np.sin(psi)) / mag_thta)

fronto_corrected = 0.5 * mag_thta * (tdef * np.cos(2 * beta) - div) * 1000 * 100 * 3600 * 3

print()
print('Frontogenesis with spherical corrections \n')
error_stats(fronto_corrected, out_var)

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


Frontogenesis 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.12352715897307137
  Mean Values (GEMPAK): 0.1227312228429547

Max Comparison
  Max Values (MetPy): 523.5554042672868
  Max Values (GEMPAK): 534.03

Min Comparison
  Min Values (MetPy): -57.57274972404531
  Min Values (GEMPAK): -58.04

Difference Array
[ -7.11903283  -8.00999097  -6.25468993 ...  -7.79906569  -8.75270442
 -12.20079428]

Various Statistical Analyses
  Average Absolute Difference: 0.03341852840231018
  RMS Error: 0.0004358868282101384
  Standard Deviation of Difference: 0.2178133352804262
  Max Diff: 12.449693594480868
  Min Diff: -13.449124116780306
  Correlation: 0.9949651552458282
  Relative Magnitude Difference: 6.382997507031609e-05



/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)



Frontogenesis with spherical corrections 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.12216784985405062
  Mean Values (GEMPAK): 0.1227312228429547

Max Comparison
  Max Values (MetPy): 531.7328686165725
  Max Values (GEMPAK): 534.03

Min Comparison
  Min Values (MetPy): -57.78941680508653
  Min Values (GEMPAK): -58.04

Difference Array
[ 0.35844805 -0.5161581   0.5481298  ...  5.30393236  4.37621331
  1.35941525]

Various Statistical Analyses
  Average Absolute Difference: 0.007872779720951482
  RMS Error: 0.00026973694817512134
  Standard Deviation of Difference: 0.134787700838486
  Max Diff: 6.87783331070357
  Min Diff: -8.79968535427397
  Correlation: 0.9980788728189927
  Relative Magnitude Difference: 1.480589255547501e-05



/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [76]:
# To Knots

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_to_knots.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

wspd = mpcalc.wind_speed(data_var_u, data_var_v)

wspd_kts = wspd.metpy.convert_units('kt')
wspd_kts = wspd * 1.9425 # Actual GEMPAK conversion

print('Knots Unit Conversion \n')
error_stats(wspd_kts, out_var)

Knots Unit Conversion 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 25.0295352935791
  Mean Values (GEMPAK): 25.029535236611267

Max Comparison
  Max Values (MetPy): 115.58544158935547
  Max Values (GEMPAK): 115.5854

Min Comparison
  Min Values (MetPy): 0.028534580022096634
  Min Values (GEMPAK): 0.0285

Difference Array
[-6.40869141e-07 -1.23809814e-05 -1.07727051e-05 ...  3.53363037e-05
  2.73681641e-05 -1.67633057e-05]

Various Statistical Analyses
  Average Absolute Difference: 2.500790969274658e-05
  RMS Error: 5.6691113419019504e-08
  Standard Deviation of Difference: 2.8896282586230948e-05
  Max Diff: 6.536865234352263e-05
  Min Diff: -5.349121093445319e-05
  Correlation: 0.9999999999988006
  Relative Magnitude Difference: 2.1635864645993285e-07



In [84]:
# Laplacian

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_laplacian.out')

out_var = gempak_out_var * 10**scale

data_var_hght = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

data_test = mpcalc.laplacian(data_var_hght.metpy.unit_array, deltas=(dy, dx))

dzdy, dzdx = mpcalc.gradient(data_var_hght, deltas=(dy, dx))
data_test2 = mpcalc.divergence(dzdx, dzdy, dx, dy)

print('Laplacian with MetPy Function \n')
error_stats(data_test[3:-3,3:-3]*10**7, out_var[3:-3,3:-3]*10**7)

print('\nLaplacian calculated the GEMPAK way \n')
error_stats(data_test2[3:-3,3:-3]*10**7, out_var[3:-3,3:-3]*10**7)

Laplacian with MetPy Function 

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): 3.28700520703842e-05
  Mean Values (GEMPAK): 0.00018666386554621852

Max Comparison
  Max Values (MetPy): 4.822491750319185
  Max Values (GEMPAK): 2.0254800000000004

Min Comparison
  Min Values (MetPy): -7.069483557956639
  Min Values (GEMPAK): -2.14576

Difference Array
[ 0.07872183 -0.06435186  0.0064418  ...  0.06373096 -0.00894591
 -0.07535852]

Various Statistical Analyses
  Average Absolute Difference: 0.007286389925609259
  RMS Error: 8.401273073408034e-05
  Standard Deviation of Difference: 0.04229660531157714
  Max Diff: 3.839420734310159
  Min Diff: -4.923723557956639
  Correlation: 0.7310089906589184
  Relative Magnitude Difference: 0.0015109180695077387


Laplacian calculated the GEMPAK way 

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): 3.325593742454068e-05
  Mean Values (GEMPAK): 0.00018666386

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:930: RuntimeWarning: invalid value encountered in subtract
  magnitude = op(self._magnitude, other._magnitude)


In [86]:
# Magnitude

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_magnitude.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

data_test = mpcalc.wind_speed(data_var_u, data_var_v)

print('Magnitude \n')
error_stats(data_test, out_var)

Magnitude 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 12.885218620300293
  Mean Values (GEMPAK): 12.885249615266236

Max Comparison
  Max Values (MetPy): 59.50344467163086
  Max Values (GEMPAK): 59.5

Min Comparison
  Min Values (MetPy): 0.014689616858959198
  Min Values (GEMPAK): 0.01

Difference Array
[ 2.35475540e-03 -4.87100601e-03 -1.73007965e-03 ...  4.41741943e-05
 -2.13474274e-03 -3.34209442e-03]

Various Statistical Analyses
  Average Absolute Difference: 0.002520989038211271
  RMS Error: 5.702845010733927e-06
  Standard Deviation of Difference: 0.0029072721124964378
  Max Diff: 0.005003662109373863
  Min Diff: -0.004999961853027202
  Correlation: 0.9999999542244651
  Relative Magnitude Difference: 4.2367110881114914e-05



In [87]:
# Mixing Ratio

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_mixing_ratio.out')

out_var = gempak_out_var * 10**scale
# Set GEMPAK missing values to 0 RH
out_var[out_var == -9999] = 0.0

data_var_rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)
data_var_tmpk = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)

mixr = mpcalc.mixing_ratio_from_relative_humidity(850*units.hPa, data_var_tmpk, data_var_rh)

print('Mixing Ratio \n')
error_stats(mixr, out_var/1000)

Mixing Ratio 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 0.004499981179833412
  Mean Values (GEMPAK): 0.004491659510618651

Max Comparison
  Max Values (MetPy): 0.02085353620350361
  Max Values (GEMPAK): 0.0209

Min Comparison
  Min Values (MetPy): 0.0
  Min Values (GEMPAK): 0.0

Difference Array
[8.55520386e-07 8.55520386e-07 8.55520386e-07 ... 3.13666604e-06
 3.13666604e-06 3.13666604e-06]

Various Statistical Analyses
  Average Absolute Difference: 1.8316640592791447e-05
  RMS Error: 5.950263234069754e-08
  Standard Deviation of Difference: 2.9172017267723182e-05
  Max Diff: 0.00032184840202331526
  Min Diff: -7.814847409725115e-05
  Correlation: 0.9999758422223188
  Relative Magnitude Difference: 0.0008783469822117777



In [88]:
# SM5S

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sm5s_filter.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

smooth_data_var = mpcalc.smooth_n_point(data_var)

print('5-point Smoothing \n')
error_stats(smooth_data_var[1:-1, 1:-1], out_var[1:-1, 1:-1])

5-point Smoothing 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 5491.81005859375
  Mean Values (GEMPAK): 5491.809413800328

Max Comparison
  Max Values (MetPy): 5917.91357421875
  Max Values (GEMPAK): 5917.91

Min Comparison
  Min Values (MetPy): 4742.19189453125
  Min Values (GEMPAK): 4742.19

Difference Array
[-0.00351562 -0.00128906  0.00410156 ... -0.00335938 -0.00335938
 -0.00361328]

Various Statistical Analyses
  Average Absolute Difference: 0.0025012574432758377
  RMS Error: 5.5421242537137824e-06
  Standard Deviation of Difference: 0.002811663079031467
  Max Diff: 0.00480468749992724
  Min Diff: -0.004765624999890861
  Correlation: 0.9999999999673687
  Relative Magnitude Difference: 4.226586637176498e-07



In [89]:
# SM9S

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sm9s_filter.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

smooth_data_var = mpcalc.smooth_n_point(data_var, 9)

print('9-point Smoothing \n')
error_stats(smooth_data_var[1:-1, 1:-1], out_var[1:-1, 1:-1])

9-point Smoothing 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 5491.8095703125
  Mean Values (GEMPAK): 5491.809602889489

Max Comparison
  Max Values (MetPy): 5917.802734375
  Max Values (GEMPAK): 5917.8

Min Comparison
  Min Values (MetPy): 4742.9501953125
  Min Values (GEMPAK): 4742.95

Difference Array
[ 0.00367187  0.00443359  0.00177734 ... -0.00238281 -0.00189453
 -0.00482422]

Various Statistical Analyses
  Average Absolute Difference: 0.0025181949276320463
  RMS Error: 5.765185658250286e-06
  Standard Deviation of Difference: 0.002923791237174214
  Max Diff: 0.007265625000400178
  Min Diff: -0.0060742187497453415
  Correlation: 0.9999999999646569
  Relative Magnitude Difference: 4.2552870392324795e-07



In [90]:
# PVOR
gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_pvor_500.out')

out_var = gempak_out_var * 10**scale

lats = gfs_data.lat.values * units.degree
lons = gfs_data.lon.values * units.degree

p = gfs_data.Geopotential_height_isobaric.isobaric2
T = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0))

uwnd = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0))
vwnd = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0))

potemp = mpcalc.potential_temperature(p, T)

pvor = mpcalc.potential_vorticity_baroclinic(potemp, p, uwnd, vwnd, latitude=lats[None, :, None])


# Begin Spherical Correction Calculation


dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

avor = (mpcalc.absolute_vorticity(uwnd, vwnd, latitude=lats[None, :, None]) 
        + uwnd.metpy.quantify() / mpconst.earth_avg_radius * np.tan(np.deg2rad(lats[None, :, None])))
dthtadp = mpcalc.first_derivative(potemp, x=p.values[:, None, None] * units.Pa, axis=-3)

if ((np.shape(potemp)[-2] == 1)
   and (np.shape(potemp)[-1] == 1)):
    dthtady = 0 * units.K / units.m  # axis=-2 only has one dimension
    dthtadx = 0 * units.K / units.m  # axis=-1 only has one dimension
else:
    dthtady = mpcalc.first_derivative(potemp, delta=dy[None, :, :], axis=-2)
    dthtadx = mpcalc.first_derivative(potemp, delta=dx[None, :, :], axis=-1)
dudp = mpcalc.first_derivative(uwnd, x=p.values[:, None, None] * units.Pa, axis=-3)
dvdp = mpcalc.first_derivative(vwnd, x=p.values[:, None, None] * units.Pa, axis=-3)

pvor_corrected = (-mpconst.g * (dudp * dthtady - dvdp * dthtadx
                       + avor * dthtadp)).data.to_base_units()

data_var = pvor[-2]

print('Baroclinic Potential Vorticity Calculation \n')
error_stats(data_var[3:-3, 3:-3]*10**6, out_var[3:-3, 3:-3]*10**6)

print()
print('Baroclinic Potential Vorticity with spherical corrections \n')
error_stats(pvor_corrected[-2, 3:-3, 3:-3]*10**6, out_var[3:-3, 3:-3]*10**6)

/Users/kgoebber/unidata_stuff/MetPy/src/metpy/calc/basic.py:1183: UserWarning: Input over 1.5707963267948966 radians. Ensure proper units are given.
  warnings.warn('Input over {} radians. '


Baroclinic Potential Vorticity Calculation 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -0.01526443568313161
  Mean Values (GEMPAK): -0.01685379950289974

Max Comparison
  Max Values (MetPy): 4.812561094146297
  Max Values (GEMPAK): 4.759060000000001

Min Comparison
  Min Values (MetPy): -9.441901375861596
  Min Values (GEMPAK): -9.367530000000002

Difference Array
[0.19923624 0.17355321 0.18698627 ... 0.42421274 0.43928798 0.45370307]

Various Statistical Analyses
  Average Absolute Difference: 0.05418500506091589
  RMS Error: 0.00017209115012498994
  Standard Deviation of Difference: 0.0866260943561178
  Max Diff: 1.331903512957956
  Min Diff: -1.2910366151153683
  Correlation: 0.9915463126298004
  Relative Magnitude Difference: 0.011259078898099226


Baroclinic Potential Vorticity with spherical corrections 

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): -0.0171717492255331
  Mean Values (GEMPAK): -0.01685

/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/quantity.py:1160: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [93]:
# Potential Temperature

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_potential_temperature.out')

out_var = gempak_out_var * 10**scale

T = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
potemp = mpcalc.potential_temperature(500 * units.hPa, T)

print('Potential Temperature MetPy Calcualation \n')
error_stats(potemp, out_var)

print()
print('Potential Temperature using GEMPAK conversion (R/Cp = 2/7) \n')
potemp2 = T * (1000./500.)**(2/7)
error_stats(potemp2, out_var)

Potential Temperature MetPy Calcualation 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 306.2960510253906
  Mean Values (GEMPAK): 306.31395891120343

Max Comparison
  Max Values (MetPy): 336.9414367675781
  Max Values (GEMPAK): 336.9611

Min Comparison
  Min Values (MetPy): 273.5564270019531
  Min Values (GEMPAK): 273.5724

Difference Array
[-0.01621465 -0.01621465 -0.01621465 ... -0.01650874 -0.01650874
 -0.01650874]

Various Statistical Analyses
  Average Absolute Difference: 0.01793505059024309
  RMS Error: 3.523608958281494e-05
  Standard Deviation of Difference: 0.0010227199653938934
  Max Diff: -0.015972998046891007
  Min Diff: -0.019731787109321886
  Correlation: 0.999999999997858
  Relative Magnitude Difference: 5.322898472298814e-05


Potential Temperature using GEMPAK conversion (R/Cp = 2/7) 

Is Xarray with Quantity? False
Is Metpy Unit Array? False
I got back just a numpy array

Mean Comparison
  Mean Values (MetPy): 306.31396484375
  Mean Values (G

In [94]:
# Relative Humidity

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_relhum.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T, rh)

data_var = mpcalc.relative_humidity_from_dewpoint(T, dwpc) * 100

print('Relative Humidity \n')
error_stats(data_var, out_var)

Relative Humidity 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 54.37925720214844
  Mean Values (GEMPAK): 54.37926151210422

Max Comparison
  Max Values (MetPy): 100.00006103515625
  Max Values (GEMPAK): 100.0

Min Comparison
  Min Values (MetPy): 0.09999984502792358
  Min Values (GEMPAK): 0.1

Difference Array
[0. 0. 0. ... 0. 0. 0.]

Various Statistical Analyses
  Average Absolute Difference: 8.185481285655828e-06
  RMS Error: 2.6450372381012438e-08
  Standard Deviation of Difference: 1.337930183862462e-05
  Max Diff: 0.00014425048827604314
  Min Diff: -0.00015035400390672748
  Correlation: 0.9999999999999264
  Relative Magnitude Difference: 8.185476289637586e-08



/Users/kgoebber/miniconda3/envs/devel_cartopy18/lib/python3.8/site-packages/pint/numpy_func.py:289: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)


In [96]:
# Max over grid

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sgmx.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

# np.nanmax takes DataArray to numpy float
data_var = np.round(np.max(hght), decimals=2)

print('Max over Grid \n')
error_stats(data_var, out_var[0, 0])

Max over Grid 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 5918.02001953125
  Mean Values (GEMPAK): 5918.02

Max Comparison
  Max Values (MetPy): 5918.02001953125
  Max Values (GEMPAK): 5918.02

Min Comparison
  Min Values (MetPy): 5918.02001953125
  Min Values (GEMPAK): 5918.02

Difference Array
[1.95312496e-05]

Various Statistical Analyses
  Average Absolute Difference: 1.9531249563442543e-05
  RMS Error: 1.9531249563442543e-05
  Standard Deviation of Difference: 0.0
  Max Diff: 1.9531249563442543e-05
  Min Diff: 1.9531249563442543e-05
  Correlation: nan
  Relative Magnitude Difference: 3.3003013675154075e-09



In [97]:
# Min over grid

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sgmn.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var = np.round(np.min(hght), decimals=2)

print('Min over grid \n')
error_stats(data_var, out_var[0, 0])

Min over grid 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 4740.919921875
  Mean Values (GEMPAK): 4740.92

Max Comparison
  Max Values (MetPy): 4740.919921875
  Max Values (GEMPAK): 4740.92

Min Comparison
  Min Values (MetPy): 4740.919921875
  Min Values (GEMPAK): 4740.92

Difference Array
[-7.81250001e-05]

Various Statistical Analyses
  Average Absolute Difference: 7.812500007275958e-05
  RMS Error: 7.812500007275958e-05
  Standard Deviation of Difference: 0.0
  Max Diff: -7.812500007275958e-05
  Min Diff: -7.812500007275958e-05
  Correlation: nan
  Relative Magnitude Difference: 1.6478869367163176e-08



In [98]:
# Saturated Potential Temperature

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_thes.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

data_var = mpcalc.saturation_equivalent_potential_temperature(500 * units.hPa, T)

print('Saturated Potential Temperature \n')
error_stats(data_var, out_var)

Saturated Potential Temperature 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 314.03033447265625
  Mean Values (GEMPAK): 314.09524428516465

Max Comparison
  Max Values (MetPy): 370.3982238769531
  Max Values (GEMPAK): 370.4918

Min Comparison
  Min Values (MetPy): 273.8619689941406
  Min Values (GEMPAK): 273.8848

Difference Array
[-0.02453408 -0.02453408 -0.02453408 ... -0.02983716 -0.02983716
 -0.02983716]

Various Statistical Analyses
  Average Absolute Difference: 0.06487981811561694
  RMS Error: 0.00013740874731523554
  Standard Deviation of Difference: 0.026423499129569324
  Max Diff: -0.022831005859359266
  Min Diff: -0.09595522460938355
  Correlation: 0.9999999890587349
  Relative Magnitude Difference: 0.00017516233592191878



In [99]:
# Equivalent Potential Temperature

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_thte.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T, rh)

data_var = mpcalc.equivalent_potential_temperature(500 * units.hPa, T, dwpc)

print('Equivalent Potential Temperature \n')
error_stats(data_var, out_var)


Equivalent Potential Temperature 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 309.3934020996094
  Mean Values (GEMPAK): 309.44015558549967

Max Comparison
  Max Values (MetPy): 365.405517578125
  Max Values (GEMPAK): 365.49980000000005

Min Comparison
  Min Values (MetPy): 273.8619689941406
  Min Values (GEMPAK): 273.8848

Difference Array
[-0.02453408 -0.02453408 -0.02453408 ... -0.02983716 -0.02983716
 -0.02983716]

Various Statistical Analyses
  Average Absolute Difference: 0.04675285931078343
  RMS Error: 0.00010082142702988857
  Standard Deviation of Difference: 0.021318010109701448
  Max Diff: -0.016752294921843713
  Min Diff: -0.09595522460938355
  Correlation: 0.9999997553663066
  Relative Magnitude Difference: 0.00012794787451666628



/Users/kgoebber/unidata_stuff/MetPy/src/metpy/calc/thermo.py:1096: RuntimeWarning: invalid value encountered in log
  t_l = 56 + 1. / (1. / (td - 56) + np.log(t / td) / 800.)


In [100]:
# Wet Bulb Temperature - output units?

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_thwc.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T.metpy.unit_array, rh.metpy.unit_array)

data_var = mpcalc.wet_bulb_temperature(1000 * units.hPa, T[25, 101], dwpc[25, 101])

print('Wet Bulb Temperature \n')
error_stats(data_var.metpy.convert_units('degC'), out_var[25, 101])
#print(np.nanmean(data_var))

Wet Bulb Temperature 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): -19.06704263276231
  Mean Values (GEMPAK): -19.07

Max Comparison
  Max Values (MetPy): -19.06704263276231
  Max Values (GEMPAK): -19.07

Min Comparison
  Min Values (MetPy): -19.06704263276231
  Min Values (GEMPAK): -19.07

Difference Array
[0.00295737]

Various Statistical Analyses
  Average Absolute Difference: 0.002957367237691244
  RMS Error: 0.002957367237691244
  Standard Deviation of Difference: 0.0
  Max Diff: 0.002957367237691244
  Min Diff: 0.002957367237691244
  Correlation: nan
  Relative Magnitude Difference: -0.0001551036148946188



In [101]:
# Moist Lapse Rate

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_tmst.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T, rh)

thte = mpcalc.equivalent_potential_temperature(500 * units.hPa, T, dwpc)

data_var = mpcalc.moist_lapse([500] * units.hPa, T[100, 100], 1000 * units.hPa)

error_stats(data_var, out_var[100, 100])
print(data_var)
print(out_var[200, 200])

Is Xarray with Quantity? False
Is Metpy Unit Array? True

Mean Comparison
  Mean Values (MetPy): 247.11064354127708
  Mean Values (GEMPAK): 244.7482

Max Comparison
  Max Values (MetPy): 247.11064354127708
  Max Values (GEMPAK): 244.7482

Min Comparison
  Min Values (MetPy): 247.11064354127708
  Min Values (GEMPAK): 244.7482

Difference Array
[2.36244354]

Various Statistical Analyses
  Average Absolute Difference: 2.362443541277088
  RMS Error: 2.362443541277088
  Standard Deviation of Difference: 0.0
  Max Diff: 2.362443541277088
  Min Diff: 2.362443541277088
  Correlation: nan
  Relative Magnitude Difference: 0.009560266233058747

247.11064354127708 kelvin
270.1614


In [105]:
# Average over row

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_xav.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var = np.round(np.mean(hght, axis=1), decimals=2)

print('Average over row \n')
error_stats(data_var, out_var[:, 0])

Average over row 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 5489.345703125
  Mean Values (GEMPAK): 5489.347174515236

Max Comparison
  Max Values (MetPy): 5859.60986328125
  Max Values (GEMPAK): 5859.61

Min Comparison
  Min Values (MetPy): 4903.35986328125
  Min Values (GEMPAK): 4903.36

Difference Array
[ 0.00000000e+00  1.17187500e-04 -1.17187500e-04 -2.14843750e-04
 -5.85937496e-05 -1.00195312e-02 -2.14843750e-04  1.75781250e-04
  1.75781250e-04  1.36718750e-04  1.75781250e-04  5.85937496e-05
 -1.02148437e-02  5.85937496e-05  5.85937496e-05  3.90625000e-05
  7.81250001e-05  1.36718750e-04 -1.17187500e-04 -5.85937496e-05
  7.81250001e-05 -1.17187500e-04 -1.75781250e-04 -1.00781250e-02
  1.00195312e-02  2.14843750e-04  9.76562496e-05  1.75781250e-04
 -1.00390625e-02 -1.75781250e-04  7.81250001e-05  9.76562496e-05
 -5.85937496e-05 -9.76562496e-05 -1.00390625e-02 -9.98046875e-03
 -2.14843750e-04 -1.95312500e-04  1.75781250e-04 -9.90234375e-03
 -1.95312496e-

In [106]:
# Average over column

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_yav.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var = np.round(np.mean(hght, axis=0), decimals=2)

print('Average over column \n')
error_stats(data_var, out_var[0, :])

Average over column 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 5489.34521484375
  Mean Values (GEMPAK): 5489.344972222222

Max Comparison
  Max Values (MetPy): 5547.93994140625
  Max Values (GEMPAK): 5547.94

Min Comparison
  Min Values (MetPy): 5457.75
  Min Values (GEMPAK): 5457.75

Difference Array
[-2.34375000e-04  9.76562496e-05  7.81250001e-05 -2.34375000e-04
  3.90625000e-05 -7.81250001e-05 -9.84375000e-03  1.75781250e-04
 -1.00195312e-02  1.95312500e-04  1.17187500e-04 -1.01757812e-02
 -1.95312500e-04 -1.17187500e-04  9.76562496e-05  9.94140625e-03
 -9.76562496e-05 -3.90625000e-05  1.95312496e-05  1.95312500e-04
 -5.85937496e-05 -7.81250001e-05  1.00585937e-02 -9.96093750e-03
  2.14843750e-04 -5.85937496e-05  9.78515625e-03 -2.14843750e-04
  1.01562500e-02 -1.95312500e-04 -9.78515625e-03 -1.95312500e-04
  1.95312500e-04  1.00585937e-02  1.56250000e-04  9.78515625e-03
  1.01171875e-02 -9.90234375e-03  1.56250000e-04 -1.75781250e-04
  9.76562496e-05  

In [107]:
# Sum over column

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_ysum.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_var = np.round(np.sum(hght, axis=0), decimals=2)

print('Sum over column')
error_stats(data_var, out_var[0, :])

Sum over column
Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 508733.25
  Mean Values (GEMPAK): 508733.2638888889

Max Comparison
  Max Values (MetPy): 520018.40625
  Max Values (GEMPAK): 520018.0

Min Comparison
  Min Values (MetPy): 497813.375
  Min Values (GEMPAK): 497813.0

Difference Array
[-3.12500000e-01  9.37500001e-02 -2.50000000e-01 -3.75000000e-01
 -9.37500000e-02  2.50000000e-01  3.12500000e-02  7.18750000e-01
 -1.87500000e-01  3.12500000e-02  2.50000000e-01  4.68750000e-01
 -2.50000000e-01  4.68750000e-01 -1.87500000e-01 -1.56250000e-01
  9.37500000e-02 -3.12500000e-01  4.68750000e-01  5.93750000e-01
  2.81250000e-01  3.12500000e-02 -4.37500000e-01 -4.37500000e-01
 -2.50000000e-01  3.75000000e-01 -5.31250000e-01 -6.25000000e-01
  5.93750000e-01  3.12499999e-02 -2.81250000e-01  4.37500000e-01
 -3.75000000e-01  1.87500000e-01  3.12500000e-01 -1.25000000e-01
 -5.93750000e-01  3.75000000e-01  6.25000000e-01 -3.12500000e-02
  0.00000000e+00 -5.31250000e-

In [108]:
# Sum over row

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_xsum.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_var = np.round(np.sum(hght, axis=1), decimals=2)

print('Sum over row \n')
error_stats(data_var, out_var[:, 0])

Sum over row 

Is Xarray with Quantity? True

Mean Comparison
  Mean Values (MetPy): 1014648.0
  Mean Values (GEMPAK): 1014648.0664819945

Max Comparison
  Max Values (MetPy): 1097762.125
  Max Values (GEMPAK): 1097763.0

Min Comparison
  Min Values (MetPy): 809060.875
  Min Values (GEMPAK): 809061.0

Difference Array
[ 5.68750000e+00 -1.25000000e-01  1.25000000e-01  7.50000000e-01
 -3.75000000e-01  1.25000000e-01  1.25000000e-01  5.00000000e-01
  6.25000000e-01 -5.00000000e-01  4.37500000e-01 -1.00000000e+00
  0.00000000e+00  5.00000000e-01 -5.62500000e-01  6.25000000e-02
 -5.00000000e-01 -6.24999999e-02 -3.75000000e-01 -6.25000000e-02
 -9.37500000e-01  1.16415322e-10 -5.00000000e-01 -2.50000000e-01
 -9.37500000e-01 -1.25000000e-01 -1.25000000e-01 -1.16415322e-10
  8.12500000e-01 -3.75000000e-01 -1.87500000e-01 -2.50000000e-01
 -1.87500000e-01  5.00000000e-01 -5.62500000e-01 -1.25000000e-01
 -2.50000000e-01 -5.00000000e-01 -6.25000000e-02  1.87500000e-01
  1.87500000e-01 -2.50000000e-